In [ ]:
import os
import pandas as pd
import re
from sqlalchemy import create_engine
from termcolor import colored
import google.generativeai as genai

# Configure Gemini API Key
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))


In [ ]:
DB_USER = os.getenv("DB_USER")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

DATABASE_URL = f"postgresql://{DB_USER}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)
df = pd.read_sql("SELECT * FROM invoices", engine)
df["invoice_date"] = pd.to_datetime(df["invoice_date"], errors="coerce")


In [ ]:
def safe_to_float(val):
    match = re.findall(r"\d+\.\d+|\d+", str(val))
    return float(match[0]) if match else None

df["total_amount_clean"] = df["total_amount"].apply(safe_to_float)

In [ ]:
model = genai.GenerativeModel(model_name="models/gemini-1.5-flash")

def nl_to_filter(user_prompt):
    system_prompt = (
        "You are a helpful assistant that translates natural language into Pandas DataFrame filter expressions. "
        "Assume the DataFrame is named 'df' and has columns: invoice_number (str), vendor_name (str), "
        "total_amount_clean (float), invoice_date (datetime). Return only the expression. Do not include code comments, outputs, or quotes."
    )
    response = model.generate_content([system_prompt, user_prompt])
    return response.text.strip()

In [ ]:
def run_nl_query(prompt):
    try:
        raw_filter = nl_to_filter(prompt)
        print(colored(f"\n🔍 Raw Model Output:\n{raw_filter}", "cyan"))

        cleaned_filter = raw_filter.replace("df[", "").replace("]", "")
        print(colored(f"\n🔍 Cleaned Filter:\n{cleaned_filter}", "blue"))

        result = df.query(cleaned_filter)
        display(result[["invoice_number", "vendor_name", "total_amount", "invoice_date"]])
        print(colored(f"\n🤖 {len(result)} result(s) found.", "green"))

    except Exception as e:
        print(colored(f"❌ Error: {e}", "red"))

In [ ]:
run_nl_query("List invoices from vendor Hayes over $500")

In [ ]:
df[["invoice_number", "vendor_name", "total_amount", "total_amount_clean"]].head(20)


In [ ]:
df[df.apply(lambda row: row.astype(str).str.contains("Hayes", case=False).any(), axis=1)]
